# Meta-analytic functional decoding
## Discrete functional decoding

In this notebook, we applied meta-analytic functional decoding using the NiMARE (https://nimare.readthedocs.io/en/stable/about.html). Specifically, we used the discrete decoding approach, which estimates the statistical association between brain regions and cognitive terms based on large-scale meta-analytic databases (e.g., Neurosynth).


In [1]:
# Import necessary modules

import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nibabel as nib
from nibabel import Nifti1Image

from nilearn import datasets, image, plotting
from nilearn.plotting import plot_roi
from nilearn.image import load_img
from nilearn.masking import _unmask_3d
from nilearn.maskers import nifti_spheres_masker

import nimare
from nimare.stats import pearson
from nimare.dataset import Dataset
from nimare.decode import discrete, continuous
from nimare.utils import get_resource_path
from nimare.extract import fetch_neurosynth
from nimare.io import convert_neurosynth_to_dataset

# print the current working directory
# print(os.getcwd())

In [2]:
# get neurosynth data
databases = nimare.extract.fetch_neurosynth(data_dir='../Data')[0]

# convert to NiMARE dataset (Note: This can take a while!)
ds = nimare.io.convert_neurosynth_to_dataset(
    coordinates_file=databases['coordinates'],
    metadata_file=databases['metadata'],
    annotations_files=databases['features']
    )

INFO:nimare.extract.utils:Dataset found in ../Data/neurosynth

INFO:nimare.extract.extract:Searching for any feature files matching the following criteria: [('data-neurosynth', 'version-7')]


File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.
File exists and overwrite is False. Skipping.


In [3]:
ds.coordinates

,id,study_id,contrast_id,x,y,z,space
1483,10022492-1,10022492,1,36.0,-58.0,52.0,mni152_2mm
1499,10022492-1,10022492,1,48.0,24.0,20.0,mni152_2mm
1498,10022492-1,10022492,1,-42.0,26.0,20.0,mni152_2mm
1497,10022492-1,10022492,1,-36.0,30.0,16.0,mni152_2mm
1496,10022492-1,10022492,1,-30.0,32.0,0.0,mni152_2mm
...,...,...,...,...,...,...,...
1479,9990082-1,9990082,1,42.0,-54.0,-21.0,mni152_2mm
1480,9990082-1,9990082,1,-36.0,-87.0,-6.0,mni152_2mm
1481,9990082-1,9990082,1,30.0,-81.0,-15.0,mni152_2mm
1467,9990082-1,9990082,1,-18.0,-60.0,54.0,mni152_2mm


## 使用cognitive atlas的术语进行解码

In [5]:
import requests
print("Fetching Cognitive Atlas terms...")
base_url = "https://www.cognitiveatlas.org/api/v-alpha"


Fetching Cognitive Atlas terms...


In [6]:

concepts = requests.get(f"{base_url}/concept?format=json").json()
tasks = requests.get(f"{base_url}/task?format=json").json()

cognitive_terms = [c['name'].lower() for c in concepts] + [t['name'].lower() for t in tasks]

print(f"Fetched {len(cognitive_terms)} cognitive terms from Cognitive Atlas.")

Fetched 1771 cognitive terms from Cognitive Atlas.


In [12]:
print("Filtering annotations to retain only cognitive terms...")
terms_to_keep = ['id'] + [
    col for col in ds.annotations.columns
    if col.split('__')[-1].lower() in cognitive_terms
]

ds.annotations = ds.annotations[terms_to_keep]
print(f"Remaining features after filtering: {len(ds.annotations.columns) - 1}")

# 🧩 关键修复部分
if 'study_id' not in ds.annotations.columns:
    ds.annotations['study_id'] = ds.annotations['id']
if 'contrast_id' not in ds.annotations.columns:
    ds.annotations['contrast_id'] = ds.annotations['id']

Filtering annotations to retain only cognitive terms...
Remaining features after filtering: 133


In [13]:
from nimare.decode import continuous

zmap_path = "/Users/ss/Documents/Self_Psych_Meta/Output/1_ALE/control_all_z_thresh.nii.gz"  # <-- 修改为你的 z-map 路径

print(f"Running continuous decoding for {zmap_path}...")


Running continuous decoding for /Users/ss/Documents/Self_Psych_Meta/Output/1_ALE/control_all_z_thresh.nii.gz...


In [14]:
# define decoder parameters
frequency_threshold = 0.001
u = 0.05
correction = "fdr_bh"

decoder = continuous.CorrelationDecoder(
    frequency_threshold=frequency_threshold,
    target_image=zmap_path
)


In [15]:
decoder.fit(ds)
decoded_df = decoder.transform()


  0%|          | 0/133 [00:19<?, ?it/s]


ValueError: No map with name '/Users/ss/Documents/Self_Psych_Meta/Output/1_ALE/control_all_z_thresh.nii.gz' found.

In [ ]:

decoder.fit(ds)
decoded_df = decoder.transform()

# Save results
out_csv = "zmap_decoding_results.csv"
decoded_df.to_csv(out_csv, index=False)
print(f"Decoding results saved to {out_csv}")

In [11]:


# Cognitive Atlas API base URL
base_url = "https://www.cognitiveatlas.org/api/v-alpha"

# Endpoints for concepts, tasks, and disorders
concepts_endpoint = f"{base_url}/concept?format=json"
tasks_endpoint = f"{base_url}/task?format=json"
disorders_endpoint = f"{base_url}/disorder?format=json"


# Fetch concepts and tasks data
concepts_response = requests.get(concepts_endpoint)
tasks_response = requests.get(tasks_endpoint)
disorders_response = requests.get(disorders_endpoint)

# Extract names from the response data
concepts = concepts_response.json()
tasks = tasks_response.json()
disorders = disorders_response.json()

# Get the names of concepts and tasks
concept_names = [concept['name'] for concept in concepts]
task_names = [task['name'] for task in tasks]
disorder_names = [disorder['name'] for disorder in disorders]

cognitive_atlas_terms = concept_names + task_names + disorder_names

SSLError: HTTPSConnectionPool(host='www.cognitiveatlas.org', port=443): Max retries exceeded with url: /api/v-alpha/concept?format=json (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1010)')))

In [ ]:
terms_to_keep = ['id'] + [
    term for term in ds.annotations.columns 
    if term.split('__')[-1].replace('_', ' ') in cognitive_atlas_terms
]

ds.annotations = ds.annotations[terms_to_keep]


In [ ]:
print(ds.annotations.columns[:10])
print(cognitive_atlas_terms[:10])


In [ ]:
# define decoder parameters
frequency_threshold = 0.001
u = 0.05
correction = "fdr_bh"

In [ ]:
from nimare.decode.continuous import CorrelationDecoder
from nimare.meta.cbma import mkda
from nilearn.image import load_img
import os

In [ ]:
ale_img_path = "/Users/ss/Documents/Self_Psych_Meta/Output/1_ALE/control_all_z_thresh.nii"



In [ ]:
decoder = CorrelationDecoder(ale_img_path,
    frequency_threshold=frequency_threshold,
    meta_estimator=mkda.MKDAChi2,
    target_image='z_desc-association'
)


In [ ]:

decoder.fit(ds)


In [ ]:

decoded_df = decoder.transform()


In [ ]:

os.makedirs('/Users/ss/Documents/Self_Psych_Meta/Output/6_Decoding', exist_ok=True)
decoded_df.to_csv('/Users/ss/Documents/Self_Psych_Meta/Output/6_Decoding/control_all_decode.csv', index=True)
# ...existing code...

In [ ]:

decoder.fit(ds)
decoded_df = decoder.transform()

In [ ]:
decoded_df.to_csv('/Users/ss/Documents/Self_Psych_Meta/Output/6_Decoding/L_Angular.csv', index=True)

### Plot word cloud

In [ ]:
from wordcloud import WordCloud
import matplotlib

# 定义输入文件列表
csv_files = [
    '../results/extract_decoding/extract_Angular_decode.csv',
    '../results/extract_decoding/extract_Cingulate_decode.csv',
    '../results/extract_decoding/extract_FPole_decode.csv',
    '../results/extract_decoding/extract_L_FOC_decode.csv',
    '../results/extract_decoding/extract_R_FOC_decode.csv',
    '../results/extract_decoding/extract_PCG_decode.csv'
]

# 定义输出目录
output_dir = '../results/wordcloud'
os.makedirs(output_dir, exist_ok=True)  # 如果目录不存在，则创建


In [ ]:
import os
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# 定义输入文件列表
csv_files = [
    '/Users/ss/Documents/Self_Psych_Meta/Output/wordcloud/control_clean.csv',
    '/Users/ss/Documents/Self_Psych_Meta/Output/wordcloud/control_minus_patient_clean.csv',
    '/Users/ss/Documents/Self_Psych_Meta/Output/wordcloud/patient_clean.csv',
    '/Users/ss/Documents/Self_Psych_Meta/Output/wordcloud/patient_minus_control_clean.csv',
    '/Users/ss/Documents/Self_Psych_Meta/Output/wordcloud/sz_clean.csv'
]

# 定义输出目录
output_dir = '/Users/ss/Documents/Self_Psych_Meta/Output/wordcloud'
os.makedirs(output_dir, exist_ok=True)  # 如果目录不存在，则创建


# 批量处理每个 CSV 文件
for file_path in csv_files:
    # 读取 CSV 数据
    data = pd.read_csv(file_path)

    # 构建词频字典，使用 'feature' 作为单词，'r' 作为词频
    word_frequencies = data.set_index('feature')['r'].to_dict()

    # 创建词云
    wordcloud = WordCloud(
        width=800,
        height=400,
        colormap="Reds",
        background_color='white',
        max_words=150,
        max_font_size=150,
        scale=3,
        relative_scaling=0.5,
        prefer_horizontal=1.0
    ).generate_from_frequencies(word_frequencies)

    # 绘制并显示词云图
    plt.figure(figsize=(10, 4))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')  # 关闭坐标轴

    # 获取'r'值的范围，用于color bar
    r_values = list(word_frequencies.values())
    vmin, vmax = min(r_values), max(r_values)

    # 创建一个标量映射对象，用于color bar
    norm = plt.Normalize(vmin=vmin, vmax=vmax)
    sm = plt.cm.ScalarMappable(cmap="Reds", norm=norm)
    sm.set_array([])  

    # 添加color bar
    cbar = plt.colorbar(sm, ax=plt.gca(), orientation='vertical', fraction=0.046, pad=0.04)
    cbar.set_label('', rotation=270, labelpad=15)  # 设置color bar的标签

    # 显示图像
    plt.show()

    # 保存词云图为文件
    file_name = os.path.basename(file_path).replace('extract_', '').replace('_decode.csv', '')  # 提取文件名的一部分
    output_path = os.path.join(output_dir, f'{file_name}_wordcloud.png')
    wordcloud.to_file(output_path)

    print(f"词云图已保存到 {output_path}")

In [ ]:
# 批量处理每个 CSV 文件
# "viridis" 绿色
for file_path in csv_files:
    # 读取 CSV 数据
    data = pd.read_csv(file_path)

    # 构建词频字典，使用 'cognitive_feature' 作为单词，'r' 作为词频
    word_frequencies = data.set_index('cognitive_feature')['r'].to_dict()

    # 创建词云
    wordcloud = WordCloud(
        width=800,
        height=400,
        colormap="YlOrRd",
        background_color='white',
        prefer_horizontal=1.0
    ).generate_from_frequencies(word_frequencies)

    # 绘制并显示词云图
    plt.figure(figsize=(8, 4))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')  # 关闭坐标轴
    plt.show()

    # 保存词云图为文件
    file_name = os.path.basename(file_path).replace('extract_', '').replace('_decode.csv', '')  # 提取文件名的一部分
    output_path = os.path.join(output_dir, f'{file_name}_wordcloud.png')
    wordcloud.to_file(output_path)

    print(f"词云图已保存到 {output_path}")
